# Codex CLI & Agents SDKを使用した一貫性のあるワークフローの構築
### 反復可能で追跡可能、そしてスケーラブルなエージェント開発の確保

## はじめに
開発者は、自分たちが行うすべてのことにおいて一貫性を求めています。Codex CLIとAgents SDKを使用することで、その一貫性をこれまでにないスケールで実現できるようになりました。大規模なコードベースのリファクタリング、新機能の展開、新しいテストフレームワークの導入など、どのような作業においても、CodexはシームレスにCLI、IDE、クラウドワークフローに統合され、反復可能な開発パターンを自動化し、強制します。

このトラックでは、Agents SDKを使用してシングルエージェントとマルチエージェントの両方のシステムを構築し、Codex CLIをMCPサーバーとして公開します。これにより以下が可能になります：
- **一貫性と反復可能性** - 各エージェントにスコープ化されたコンテキストを提供することで実現
- **スケーラブルなオーケストレーション** - シングルエージェントとマルチエージェントシステムを調整
- **可観測性と監査可能性** - 完全なエージェントスタックトレースを確認することで実現

## 取り上げる内容
- Codex CLIをMCPサーバーとして初期化：Codexを長時間実行されるMCPプロセスとして実行する方法
- シングルエージェントシステムの構築：スコープ化されたタスクにCodex MCPを使用
- マルチエージェントワークフローのオーケストレーション：複数の専門エージェントの調整
- エージェント動作の追跡：可視性と評価のためのエージェントトレースの活用

## 前提条件とセットアップ
このトラックを開始する前に、以下を確認してください：
- 基本的なコーディング知識：PythonとJavaScriptに慣れている必要があります
- 開発環境：VS CodeやCursorなどのIDEが必要です
- OpenAI APIキー：OpenAI DashboardでAPIキーを作成または確認してください

## 環境セットアップ
1. ディレクトリに`.env`フォルダを作成し、`OPENAI_API_KEY`キーを追加します
2. 依存関係をインストールします

In [ ]:
%pip install openai-agents openai ## install dependencies

## Codex CLIをMCPサーバーとして初期化する
ここでは、Agents SDK内でCodex CLIをMCPサーバーとして実行します。`codex mcp`の初期化パラメータを提供します。このコマンドはCodex CLIをMCPサーバーとして開始し、MCPサーバー上で利用可能な2つのCodexツール — `codex()`と`codex-reply()` — を公開します。これらは、Agents SDKがCodexを呼び出す必要があるときに呼び出す基盤となるツールです。
- `codex()`は会話を作成するために使用されます。
- `codex-reply()`は会話を継続するために使用されます。

```python
import asyncio
from agents import Agent, Runner
from agents.mcp import MCPServerStdio

async def main() -> None:
    async with MCPServerStdio(
        name="Codex CLI",
        params={
            "command": "npx",
            "args": ["-y", "codex", "mcp-server"],
        },
        client_session_timeout_seconds=360000,
    ) as codex_mcp_server:
        print("Codex MCP server started.")
        # 次のセクションでここにさらにコードを追加します
        return
```

また、Codex CLIが与えられたタスクを実行して完了するのに十分な時間を確保するため、MCPサーバーのタイムアウトを延長していることに注意してください。

---

## 単一エージェントシステムの構築
Codex MCPサーバーを使用する簡単な例から始めましょう。2つのエージェントを定義します：
1. **デザイナーエージェント** – ゲームのアイデアを考案し、簡単な仕様書を作成します。
2. **開発者エージェント** – デザイナーの仕様に従って簡単なゲームを実装します。

```python
developer_agent = Agent(
    name="Game Developer",
    instructions=(
        "You are an expert in building simple games using basic html + css + javascript with no dependencies. "
        "Save your work in a file called index.html in the current directory."
        "Always call codex with \"approval-policy\": \"never\" and \"sandbox\": \"workspace-write\""
    ),
    mcp_servers=[codex_mcp_server],
)

designer_agent = Agent(
    name="Game Designer",
    instructions=(
        "You are an indie game connoisseur. Come up with an idea for a single page html + css + javascript game that a developer could build in about 50 lines of code. "
        "Format your request as a 3 sentence design brief for a game developer and call the Game Developer coder with your idea."
    ),
    model="gpt-5",
    handoffs=[developer_agent],
)

result = await Runner.run(designer_agent, "Implement a fun new game!")
```

開発者エージェントに、ユーザーの許可を求めることなくプロジェクトディレクトリにファイルを書き込む能力を提供していることに注意してください。

コードを実行すると、`index.html`ファイルが生成されます。ファイルを開いてゲームをプレイしてみてください！

以下は、私のエージェントシステムが作成したゲームのスクリーンショットです。あなたのものは異なるでしょう！

| ゲームプレイ例 | ゲームオーバースコア |
| :---: | :---: |
| <img src="/images/game_example_1.png" alt="ゲームプレイ例" width="320" /> | <img src="/images/game_example_2.png" alt="ゲームオーバースコア" width="320" /> |

以下が実行可能な完全なコードです。実行には数分かかる場合があることに注意してください。`index.html`ファイルが生成されれば、正常に実行されたことになります。また、フォーマットに関するMCPイベントの警告が表示される場合がありますが、これらのイベントは無視して構いません。

In [ ]:
import os
from dotenv import load_dotenv
import asyncio
from agents import Agent, Runner, set_default_openai_api
from agents.mcp import MCPServerStdio

load_dotenv(override=True) # load the API key from the .env file. We set override to True here to ensure the notebook is loading any changes
set_default_openai_api(os.getenv("OPENAI_API_KEY"))

async def main() -> None:
    async with MCPServerStdio(
        name="Codex CLI",
        params={
            "command": "npx",
            "args": ["-y", "codex", "mcp-server"],
        },
        client_session_timeout_seconds=360000,
    ) as codex_mcp_server:
        developer_agent = Agent(
            name="Game Developer",
            instructions=(
                "You are an expert in building simple games using basic html + css + javascript with no dependencies. "
                "Save your work in a file called index.html in the current directory."
                "Always call codex with \"approval-policy\": \"never\" and \"sandbox\": \"workspace-write\""
            ),
            mcp_servers=[codex_mcp_server],
        )

        designer_agent = Agent(
            name="Game Designer",
            instructions=(
                "You are an indie game connoisseur. Come up with an idea for a single page html + css + javascript game that a developer could build in about 50 lines of code. "
                "Format your request as a 3 sentence design brief for a game developer and call the Game Developer coder with your idea."
            ),
            model="gpt-5",
            handoffs=[developer_agent],
        )

        result = await Runner.run(designer_agent, "Implement a fun new game!")
        # print(result.final_output)


if __name__ == "__main__":
    # Jupyter/IPython already runs an event loop, so calling asyncio.run() here
    # raises "asyncio.run() cannot be called from a running event loop".
    # Workaround: if a loop is running (notebook), use top-level `await`; otherwise use asyncio.run().
    try:
        asyncio.get_running_loop()
        await main()
    except RuntimeError:
        asyncio.run(main())

---

## マルチエージェントワークフローのオーケストレーション
より大規模なワークフローのために、エージェントのチームを導入します：
- **プロジェクトマネージャー**: タスクリストを分解し、要件を作成し、作業を調整します。
- **デザイナー**: UI/UX仕様を作成します。
- **フロントエンド開発者**: UI/UXを実装します。
- **バックエンド開発者**: APIとロジックを実装します。
- **テスター**: 受け入れ基準に対してアウトプットを検証します。

この例では、プロジェクトマネージャーエージェントが各専門的な下流エージェント間でゲート制御ロジックを意図的に実行させています。これにより、引き継ぎが行われる前に成果物が存在することを保証します。これは、JIRAタスクオーケストレーション、長期間のロールアウト、QAサインオフなどの実世界のエンタープライズワークフローを反映しています。

<div align="center">
  <img src="/images/multi_agent_codex_workflow.png" alt="Multi-Agent Codex Workflow with Codex MCP" style="max-width: 100%; width: 960px;" />
  <br />
  <em>Codex MCPとゲート制御された引き継ぎによる成果物生成を伴うマルチエージェントオーケストレーション。</em>
</div>

この構造では、各エージェントが専門的な目的を果たします。プロジェクトマネージャーは、他のすべてのエージェント間の調整と全体的なタスクの完了を確実にする全体的な責任を負います。

## Codex CLI MCPサーバーの定義
シングルエージェントの例で行ったように、Codex CLIを初期化するためのMCPサーバーを設定します。

```python
async def main() -> None:
    async with MCPServerStdio(
        name="Codex CLI",
        params={
            "command": "npx",
            "args": ["-y", "codex", "mcp-server"],
        },
        client_session_timeout_seconds=360000,
    ) as codex_mcp_server:
        print("Codex MCP server started.")
        # 次のセクションでここにさらにコードを追加します
        return
 ```

## 各専門エージェントの定義
以下では、各専門エージェントを定義し、Codex MCPサーバーへのアクセスを提供します。また、エージェント間の引き継ぎを最適化するためにシステムを支援する`RECOMMMENDED_PROMPT_PREFIX`を各エージェントに渡していることにも注目してください。

```python
# 明確性のために下流エージェントを最初に定義し、その後PMが引き継ぎでそれらを参照します。
designer_agent = Agent(
    name="Designer",
    instructions=(
        f"""{RECOMMENDED_PROMPT_PREFIX}"""
        "あなたはデザイナーです。\n"
        "あなたの唯一の情報源は、プロジェクトマネージャーからのAGENT_TASKS.mdとREQUIREMENTS.mdです。\n"
        "そこに書かれていないことは何も仮定しないでください。\n\n"
        "追加のガイダンスや調査のためにインターネットを使用することができます。"
        "成果物（/designに書き込み）：\n"
        "- design_spec.md – AGENT_TASKS.mdで要求されたUI/UXレイアウト、メイン画面、主要な視覚的注記を説明する単一ページ。\n"
        "- wireframe.md – 指定された場合のシンプルなテキストまたはASCIIワイヤーフレーム。\n\n"
        "アウトプットは短く、実装に適したものにしてください。\n"
        "完了時は、transfer_to_project_managerでプロジェクトマネージャーに引き継いでください。"
        "ファイルを作成する際は、{\"approval-policy\":\"never\",\"sandbox\":\"workspace-write\"}でCodex MCPを呼び出してください。"
    ),
    model="gpt-5",
    tools=[WebSearchTool()],
    mcp_servers=[codex_mcp_server],
    handoffs=[],
)

frontend_developer_agent = Agent(
    name="Frontend Developer",
    instructions=(
        f"""{RECOMMENDED_PROMPT_PREFIX}"""
        "あなたはフロントエンド開発者です。\n"
        "AGENT_TASKS.mdとdesign_spec.mdを読んでください。そこに記述されたとおりに正確に実装してください。\n\n"
        "成果物（/frontendに書き込み）：\n"
        "- index.html – メインページ構造\n"
        "- styles.cssまたは指定された場合のインラインスタイル\n"
        "- 指定された場合のmain.jsまたはgame.js\n\n"
        "デザイナーのDOM構造とプロジェクトマネージャーから提供された統合ポイントに従ってください。\n"
        "提供された文書を超えた機能やブランディングは追加しないでください。\n\n"
        "完了時は、transfer_to_project_manager_agentでプロジェクトマネージャーに引き継いでください。"
        "ファイルを作成する際は、{\"approval-policy\":\"never\",\"sandbox\":\"workspace-write\"}でCodex MCPを呼び出してください。"
    ),
    model="gpt-5",
    mcp_servers=[codex_mcp_server],
    handoffs=[],
)

backend_developer_agent = Agent(
    name="Backend Developer",
    instructions=(
        f"""{RECOMMENDED_PROMPT_PREFIX}"""
        "あなたはバックエンド開発者です。\n"
        "AGENT_TASKS.mdとREQUIREMENTS.mdを読んでください。そこに記述されたバックエンドエンドポイントを実装してください。\n\n"
        "成果物（/backendに書き込み）：\n"
        "- package.json – 要求された場合はstartスクリプトを含める\n"
        "- server.js – 指定されたとおりにAPIエンドポイントとロジックを正確に実装\n\n"
        "コードはできるだけシンプルで読みやすくしてください。外部データベースは不要です。\n\n"
        "完了時は、transfer_to_project_manager_agentでプロジェクトマネージャーに引き継いでください。"
        "ファイルを作成する際は、{\"approval-policy\":\"never\",\"sandbox\":\"workspace-write\"}でCodex MCPを呼び出してください。"
    ),
    model="gpt-5",
    mcp_servers=[codex_mcp_server],
    handoffs=[],
)

tester_agent = Agent(
    name="Tester",
    instructions=(
        f"""{RECOMMENDED_PROMPT_PREFIX}"""
        "あなたはテスターです。\n"
        "AGENT_TASKS.mdとTEST.mdを読んでください。他の役割のアウトプットが受け入れ基準を満たしていることを確認してください。\n\n"
        "成果物（/testsに書き込み）：\n"
        "- TEST_PLAN.md – 要求されたマニュアルチェックまたは自動化ステップの箇条書きリスト\n"
        "- 指定された場合のtest.shまたはシンプルな自動化スクリプト\n\n"
        "最小限で実行しやすくしてください。\n\n"
        "完了時は、transfer_to_project_managerでプロジェクトマネージャーに引き継いでください。"
        "ファイルを作成する際は、{\"approval-policy\":\"never\",\"sandbox\":\"workspace-write\"}でCodex MCPを呼び出してください。"
    ),
    model="gpt-5",
    mcp_servers=[codex_mcp_server],
    handoffs=[],
)
```

各役割が割り当てを完了した後、`transfer_to_project_manager_agent`を呼び出し、プロジェクトマネージャーが必要なファイルが存在することを確認（または修正を要求）してから次のチームのブロックを解除します。

## プロジェクトマネージャーエージェントの定義
プロジェクトマネージャーは、初期プロンプトを受け取り、プロジェクトディレクトリに計画文書を作成し、すべての転送前にゲートキーピングロジックを実行する唯一のエージェントです。

```python        
project_manager_agent = Agent(
name="Project Manager",
instructions=(
    f"""{RECOMMENDED_PROMPT_PREFIX}"""
    """
    あなたはプロジェクトマネージャーです。

    目的：
    入力タスクリストを、チームが実行する3つのプロジェクトルートファイルに変換します。

    成果物（プロジェクトルートに書き込み）：
    - REQUIREMENTS.md: 製品目標、対象ユーザー、主要機能、制約の簡潔な要約。
    - TEST.md: [Owner]タグ（Designer、Frontend、Backend、Tester）と明確な受け入れ基準を持つタスク。
    - AGENT_TASKS.md: 役割ごとに1つのセクションを含む：
        - プロジェクト名
        - 必要な成果物（正確なファイル名と目的）
        - 主要な技術的注記と制約

    プロセス：
    - 最小限で合理的な仮定で曖昧さを解決します。各役割が推測せずに行動できるよう具体的にしてください。
    - {"approval-policy":"never","sandbox":"workspace-write"}でCodex MCPを使用してファイルを作成してください。
    - フォルダは作成しないでください。REQUIREMENTS.md、TEST.md、AGENT_TASKS.mdのみを作成してください。

    引き継ぎ（必要なファイルによってゲート制御）：
    1) 上記の3つのファイルが作成された後、transfer_to_designer_agentでデザイナーに引き継ぎ、REQUIREMENTS.mdとAGENT_TASKS.mdを含めてください。
    2) デザイナーが/design/design_spec.mdを作成するまで待ちます。続行する前にそのファイルが存在することを確認してください。
    3) design_spec.mdが存在する場合、以下の両方に並行して引き継ぎます：
        - transfer_to_frontend_developer_agentでフロントエンド開発者（design_spec.md、REQUIREMENTS.md、AGENT_TASKS.mdを提供）。
        - transfer_to_backend_developer_agentでバックエンド開発者（REQUIREMENTS.md、AGENT_TASKS.mdを提供）。
    4) フロントエンドが/frontend/index.htmlを、バックエンドが/backend/server.jsを作成するまで待ちます。両方のファイルが存在することを確認してください。
    5) 両方が存在する場合、transfer_to_tester_agentでテスターに引き継ぎ、すべての事前成果物とアウトプットを提供してください。
    6) そのステップに必要なファイルが存在するまで次の引き継ぎに進まないでください。何かが不足している場合は、所有エージェントに提供を要求し、再確認してください。

    PM責任：
    - すべての役割を調整し、ファイル完了を追跡し、上記のゲートチェックを実行します。
    - ステータス更新で応答しないでください。プロジェクトが完了するまで次のエージェントに引き継ぐだけです。
    """
),
model="gpt-5",
model_settings=ModelSettings(
    reasoning=Reasoning(effort="medium")
),
handoffs=[designer_agent, frontend_developer_agent, backend_developer_agent, tester_agent],
mcp_servers=[codex_mcp_server],
)
```

プロジェクトマネージャーを構築した後、スクリプトはすべてのスペシャリストの引き継ぎをプロジェクトマネージャーに戻すように設定します。これにより、次に進む前に成果物が検証のために戻されることが保証されます。

```python
designer_agent.handoffs = [project_manager_agent]
frontend_developer_agent.handoffs = [project_manager_agent]
backend_developer_agent.handoffs = [project_manager_agent]
tester_agent.handoffs = [project_manager_agent]
```

## タスクリストの追加
これは、プロジェクトマネージャーがシステム全体の具体的な要件とタスクに精緻化するタスクです。

```python
task_list = """
目標: マルチエージェントワークフローを紹介する小さなブラウザゲームを構築する。

高レベル要件：
- "Bug Busters"と呼ばれるシングルスクリーンゲーム。
- プレイヤーは動く虫をクリックしてポイントを獲得。
- ゲームは20秒後に終了し、最終スコアを表示。
- オプション: シンプルなバックエンドにスコアを送信し、トップ10リーダーボードを表示。

役割：
- デザイナー: 1ページのUI/UX仕様と基本的なワイヤーフレームを作成。
- フロントエンド開発者: ページとゲームロジックを実装。
- バックエンド開発者: 最小限のAPI（GET /health、GET/POST /scores）を実装。
- テスター: 簡単なテスト計画とコアルートを検証するシンプルなスクリプトを作成。

制約：
- 外部データベースなし—メモリストレージで十分。
- 初心者にとって読みやすくすること；フレームワークは不要。
- すべてのアウトプットは明確に名前付けされたフォルダに保存される小さなファイルであること。
"""
```

次に、システムを実行し、リラックスして座っていれば、エージェントが作業を開始し、数分でゲームを作成するのを見ることができます！以下に完全に実行可能なコードを含めています。完了すると、以下のファイルディレクトリの作成に気づくでしょう。このマルチエージェントオーケストレーションは通常、完全に完了するまでに約11分かかることに注意してください。

```markdown
root_directory/
├── AGENT_TASKS.md
├── REQUIREMENTS.md
├── backend
│   ├── package.json
│   └── server.js
├── design
│   ├── design_spec.md
│   └── wireframe.md
├── frontend
│   ├── game.js
│   ├── index.html
│   └── styles.css
└── TEST.md
```

`node server.js`でバックエンドサーバーを起動し、`index.html`ファイルを開いてゲームをプレイしてください。

In [ ]:
import os
from dotenv import load_dotenv
import asyncio
from agents import Agent, Runner, WebSearchTool, ModelSettings, set_default_openai_api
from agents.mcp import MCPServerStdio
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX
from openai.types.shared import Reasoning

load_dotenv(override=True) # load the API key from the .env file. We set override to True here to ensure the notebook is loading any changes
set_default_openai_api(os.getenv("OPENAI_API_KEY"))

async def main() -> None:
    async with MCPServerStdio(
        name="Codex CLI",
        params={"command": "npx", "args": ["-y", "codex", "mcp-server"]},
        client_session_timeout_seconds=360000,
    ) as codex_mcp_server:

        # Downstream agents are defined first for clarity, then PM references them in handoffs.
        designer_agent = Agent(
            name="Designer",
            instructions=(
                f"""{RECOMMENDED_PROMPT_PREFIX}"""
                "You are the Designer.\n"
                "Your only source of truth is AGENT_TASKS.md and REQUIREMENTS.md from the Project Manager.\n"
                "Do not assume anything that is not written there.\n\n"
                "You may use the internet for additional guidance or research."
                "Deliverables (write to /design):\n"
                "- design_spec.md – a single page describing the UI/UX layout, main screens, and key visual notes as requested in AGENT_TASKS.md.\n"
                "- wireframe.md – a simple text or ASCII wireframe if specified.\n\n"
                "Keep the output short and implementation-friendly.\n"
                "When complete, handoff to the Project Manager with transfer_to_project_manager."
                "When creating files, call Codex MCP with {\"approval-policy\":\"never\",\"sandbox\":\"workspace-write\"}."
            ),
            model="gpt-5",
            tools=[WebSearchTool()],
            mcp_servers=[codex_mcp_server],
            handoffs=[],
        )

        frontend_developer_agent = Agent(
            name="Frontend Developer",
            instructions=(
                f"""{RECOMMENDED_PROMPT_PREFIX}"""
                "You are the Frontend Developer.\n"
                "Read AGENT_TASKS.md and design_spec.md. Implement exactly what is described there.\n\n"
                "Deliverables (write to /frontend):\n"
                "- index.html – main page structure\n"
                "- styles.css or inline styles if specified\n"
                "- main.js or game.js if specified\n\n"
                "Follow the Designer’s DOM structure and any integration points given by the Project Manager.\n"
                "Do not add features or branding beyond the provided documents.\n\n"
                "When complete, handoff to the Project Manager with transfer_to_project_manager_agent."
                "When creating files, call Codex MCP with {\"approval-policy\":\"never\",\"sandbox\":\"workspace-write\"}."
            ),
            model="gpt-5",
            mcp_servers=[codex_mcp_server],
            handoffs=[],
        )

        backend_developer_agent = Agent(
            name="Backend Developer",
            instructions=(
                f"""{RECOMMENDED_PROMPT_PREFIX}"""
                "You are the Backend Developer.\n"
                "Read AGENT_TASKS.md and REQUIREMENTS.md. Implement the backend endpoints described there.\n\n"
                "Deliverables (write to /backend):\n"
                "- package.json – include a start script if requested\n"
                "- server.js – implement the API endpoints and logic exactly as specified\n\n"
                "Keep the code as simple and readable as possible. No external database.\n\n"
                "When complete, handoff to the Project Manager with transfer_to_project_manager_agent."
                "When creating files, call Codex MCP with {\"approval-policy\":\"never\",\"sandbox\":\"workspace-write\"}."
            ),
            model="gpt-5",
            mcp_servers=[codex_mcp_server],
            handoffs=[],
        )

        tester_agent = Agent(
            name="Tester",
            instructions=(
                f"""{RECOMMENDED_PROMPT_PREFIX}"""
                "You are the Tester.\n"
                "Read AGENT_TASKS.md and TEST.md. Verify that the outputs of the other roles meet the acceptance criteria.\n\n"
                "Deliverables (write to /tests):\n"
                "- TEST_PLAN.md – bullet list of manual checks or automated steps as requested\n"
                "- test.sh or a simple automated script if specified\n\n"
                "Keep it minimal and easy to run.\n\n"
                "When complete, handoff to the Project Manager with transfer_to_project_manager."
                "When creating files, call Codex MCP with {\"approval-policy\":\"never\",\"sandbox\":\"workspace-write\"}."
            ),
            model="gpt-5",
            mcp_servers=[codex_mcp_server],
            handoffs=[],
        )

        project_manager_agent = Agent(
            name="Project Manager",
            instructions=(
                f"""{RECOMMENDED_PROMPT_PREFIX}"""
                """
                You are the Project Manager.

                Objective:
                Convert the input task list into three project-root files the team will execute against.

                Deliverables (write in project root):
                - REQUIREMENTS.md: concise summary of product goals, target users, key features, and constraints.
                - TEST.md: tasks with [Owner] tags (Designer, Frontend, Backend, Tester) and clear acceptance criteria.
                - AGENT_TASKS.md: one section per role containing:
                  - Project name
                  - Required deliverables (exact file names and purpose)
                  - Key technical notes and constraints

                Process:
                - Resolve ambiguities with minimal, reasonable assumptions. Be specific so each role can act without guessing.
                - Create files using Codex MCP with {"approval-policy":"never","sandbox":"workspace-write"}.
                - Do not create folders. Only create REQUIREMENTS.md, TEST.md, AGENT_TASKS.md.

                Handoffs (gated by required files):
                1) After the three files above are created, hand off to the Designer with transfer_to_designer_agent and include REQUIREMENTS.md, and AGENT_TASKS.md.
                2) Wait for the Designer to produce /design/design_spec.md. Verify that file exists before proceeding.
                3) When design_spec.md exists, hand off in parallel to both:
                   - Frontend Developer with transfer_to_frontend_developer_agent (provide design_spec.md, REQUIREMENTS.md, AGENT_TASKS.md).
                   - Backend Developer with transfer_to_backend_developer_agent (provide REQUIREMENTS.md, AGENT_TASKS.md).
                4) Wait for Frontend to produce /frontend/index.html and Backend to produce /backend/server.js. Verify both files exist.
                5) When both exist, hand off to the Tester with transfer_to_tester_agent and provide all prior artifacts and outputs.
                6) Do not advance to the next handoff until the required files for that step are present. If something is missing, request the owning agent to supply it and re-check.

                PM Responsibilities:
                - Coordinate all roles, track file completion, and enforce the above gating checks.
                - Do NOT respond with status updates. Just handoff to the next agent until the project is complete.
                """
            ),
            model="gpt-5",
            model_settings=ModelSettings(
                reasoning=Reasoning(effort="medium")
            ),
            handoffs=[designer_agent, frontend_developer_agent, backend_developer_agent, tester_agent],
            mcp_servers=[codex_mcp_server],
        )

        designer_agent.handoffs = [project_manager_agent]
        frontend_developer_agent.handoffs = [project_manager_agent]
        backend_developer_agent.handoffs = [project_manager_agent]
        tester_agent.handoffs = [project_manager_agent]

        # Example task list input for the Project Manager
        task_list = """
Goal: Build a tiny browser game to showcase a multi-agent workflow.

High-level requirements:
- Single-screen game called "Bug Busters".
- Player clicks a moving bug to earn points.
- Game ends after 20 seconds and shows final score.
- Optional: submit score to a simple backend and display a top-10 leaderboard.

Roles:
- Designer: create a one-page UI/UX spec and basic wireframe.
- Frontend Developer: implement the page and game logic.
- Backend Developer: implement a minimal API (GET /health, GET/POST /scores).
- Tester: write a quick test plan and a simple script to verify core routes.

Constraints:
- No external database—memory storage is fine.
- Keep everything readable for beginners; no frameworks required.
- All outputs should be small files saved in clearly named folders.
"""

        # Only the Project Manager receives the task list directly
        result = await Runner.run(project_manager_agent, task_list, max_turns=30)
        print(result.final_output)

if __name__ == "__main__":
    # Jupyter/IPython already runs an event loop, so calling asyncio.run() here
    # raises "asyncio.run() cannot be called from a running event loop".
    # Workaround: if a loop is running (notebook), use top-level `await`; otherwise use asyncio.run().
    try:
        asyncio.get_running_loop()
        await main()
    except RuntimeError:
        asyncio.run(main())

---

## Tracesを使用したエージェントの動作の追跡
エージェントシステムの複雑さが増すにつれて、これらのエージェントがどのように相互作用しているかを確認することが重要になります。これは以下の内容を記録するTracesダッシュボードで行うことができます：
- プロンプト、ツール呼び出し、エージェント間のハンドオフ
- MCP Serverの呼び出し、Codex CLIの呼び出し、実行時間、ファイル書き込み
- エラーと警告

上記のエージェントチームのエージェントトレースを見てみましょう。

<div align="center">
  <img src="/images/multi_agent_trace.png" alt="Multi-Agent Codex Workflow with Codex MCP" style="max-width: 100%; width: 960px;" />
</div>

このTraceでは、すべてのエージェントハンドオフが、次のエージェントにハンドオフする前に特定のアーティファクトが存在することを確認するProject Manager Agentによって統括されていることを確認できます。さらに、Codex MCP Serverの具体的な革新と、Responses APIを呼び出すことで各出力を生成していることも確認できます。タイムラインバーは実行時間をハイライト表示し、長時間実行されるステップを簡単に特定し、エージェント間でどのように制御が渡されるかを理解できるようにします。

各トレースをクリックして、プロンプト、ツール呼び出し、その他のメタデータの具体的な詳細を確認することもできます。時間の経過とともに、この情報を表示してエージェントシステムのパフォーマンスをさらに調整、最適化、追跡することができます。

<div align="center">
  <img src="/images/multi_agent_trace_details.png" alt="Multi-Agent Trace Details" style="max-width: 100%; width: 960px;" />
</div>

---

## このガイドで行ったことの振り返り
このガイドでは、Codex CLIとAgents SDKを使用して一貫性があり、スケーラブルなワークフローを構築するプロセスを説明しました。具体的には、以下の内容をカバーしました：

- **Codex MCPサーバーのセットアップ** – Codex CLIをMCPサーバーとして初期化し、エージェントのインタラクション用のツールとして利用可能にする方法。
- **単一エージェントの例** – DesignerエージェントとDeveloperエージェントによるシンプルなワークフロー。Codexがスコープされたタスクを決定論的に実行し、プレイ可能なゲームを作成。
- **マルチエージェントオーケストレーション** – Project Manager、Designer、Frontend Developer、Backend Developer、Testerを含むより大規模なワークフローへの拡張。複雑なタスクオーケストレーションと承認プロセスを反映。
- **トレースと可観測性** – 組み込みのTracesを使用してプロンプト、ツール呼び出し、ハンドオフ、実行時間、アーティファクトをキャプチャし、デバッグ、評価、将来の最適化のためにエージェントの動作を完全に可視化。

---

## 今後の展開：これらの学習内容の適用
Codex MCPとAgents SDKの動作を確認したので、実際のプロジェクトでこれらの概念を適用し、価値を抽出する方法をご紹介します：

### 1. 実世界での展開へのスケール
- 大規模なコードリファクタリング（例：500以上のファイル、フレームワーク移行）に同じマルチエージェントオーケストレーションを適用。
- 長時間実行される監査可能な展開において、Codex MCPの決定論的実行を使用し、追跡可能な進捗を実現。

### 2. 制御を失うことなく配信を加速
- 専門化されたエージェントのチームを組織して開発を並列化し、アーティファクト検証のためのゲーティングロジックを維持。
- 新機能、テスト、またはコードベースの近代化のターンアラウンドタイムを短縮。

### 3. 開発ワークフローへの拡張と接続
- MCP対応エージェントをJira、GitHub、またはCI/CDパイプラインとwebhook経由で接続し、自動化された反復可能な開発サイクルを実現。
- マルチエージェントサービスオーケストレーションでCodex MCPを活用：コード生成だけでなく、ドキュメント作成、QA、デプロイメントにも対応。